In [ ]:
import os

import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

from utils import load_data_to_mem, augmentation, show_images


In [ ]:
train_path = os.path.join(".", "data", "train")
valid_path = os.path.join(".", "data", "val")
test_path = os.path.join(".", "data", "test")

classes = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII"]
class_mapping = {"I": 0, "II": 1, "III": 2, "IV": 3,
                 "V": 4, "VI": 5, "VII": 6, "VIII":7}
n_classes = len(classes)

def class_to_number(n):
    return class_mapping[n]

img_rows, img_cols = 64, 64


In [ ]:
X_train, y_train = load_data_to_mem(train_path, classes)
X_valid, y_valid = load_data_to_mem(valid_path, classes)
X_test, y_test = load_data_to_mem(test_path, classes)

X_train, y_train = augmentation(X_train, y_train, n_transform=1)


In [ ]:
X_train = np.array(X_train, dtype=np.float32)
X_valid = np.array(X_valid, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

# expand dimensions for CNN
X_train = np.expand_dims(X_train, axis=3) 
X_valid = np.expand_dims(X_valid, axis=3)
X_test = np.expand_dims(X_test, axis=3)

# expand dimensions for MultiLabelBinarizer
y_train = np.expand_dims(y_train, axis=1)
y_valid = np.expand_dims(y_valid, axis=1)
y_test = np.expand_dims(y_test, axis=1)

mlb = MultiLabelBinarizer()
mlb.fit(y_train)
Y_train = mlb.transform(y_train)
Y_valid = mlb.transform(y_valid)
Y_test = mlb.transform(y_test)


In [ ]:
print("Check shapes: ")
print("X_train =", X_train.shape, " Y_train =", Y_train.shape)
print("X_valid =", X_valid.shape, " Y_valid =", Y_valid.shape)
print("X_test =", X_test.shape, " Y_test =", Y_test.shape)


### Creating CNN1


In [ ]:
training_iters = 5
learning_rate = 0.001 
batch_size = 128
n_input = 100


In [ ]:
tf.reset_default_graph()

x = tf.placeholder("float", [None, img_rows, img_cols, 1])
y = tf.placeholder("float", [None, n_classes])

weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,n_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(8), initializer=tf.contrib.layers.xavier_initializer()),
}

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

def conv_net(x, weights, biases):  
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=4)
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=4)
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    # fc1 = tf.reshape(conv3, [-1, 2048])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1) 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [ ]:
pred = conv_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [ ]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [ ]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    # summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    # saver = tf.train.import_meta_graph('checkpoint/model.ckpt.meta')
    # saver.restore(sess, tf.train.latest_checkpoint('./'))
    for i in range(training_iters):
        for batch in range(len(X_train)//batch_size):
            batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
            batch_y = Y_train[batch*batch_size:min((batch+1)*batch_size,len(Y_train))]    
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: X_test, y : Y_test})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    save_path = saver.save(sess, "saved_models/model.ckpt")
    summary_writer.close()
    pred = sess.run([accuracy,cost], feed_dict={x: X_test, y : Y_test})
    print(pred)
    